In [21]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from helper import DateToOrdinal

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score

import xgboost as xgb


In [22]:
data_folder = "data"
filename = "train.csv"
df_train = pd.read_csv(os.path.join("..", data_folder, filename))#, parse_dates=True, index_col="date")

In [23]:
def create_lagged_features(df, value, lags):
    for lag in range(1, lags + 1):
        df[f't-{lag}'] = df[value].shift(lag)
    df.dropna(inplace=True)
    return df

In [24]:
df_train = df_train[["orders", "date", "warehouse"]]

In [25]:
df_lagged = create_lagged_features(df_train, "orders", lags=14)
df_lagged.tail()

,orders,date,warehouse,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14
7335,6733.0,2024-03-10,Budapest_1,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0,6322.0,6363.0
7336,6492.0,2024-03-11,Budapest_1,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0,6322.0
7337,6661.0,2024-03-12,Budapest_1,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0,6213.0
7338,6843.0,2024-03-13,Budapest_1,6661.0,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0,6340.0
7339,8130.0,2024-03-14,Budapest_1,6843.0,6661.0,6492.0,6733.0,7099.0,7488.0,6530.0,6557.0,6575.0,6450.0,6258.0,7103.0,7249.0,6443.0


In [26]:
filename1 = "test.csv"
df_test = pd.read_csv(os.path.join("..", data_folder, filename1))#, parse_dates=True, index_col="date")

df_test = df_test[["date", "warehouse"]]


In [27]:
warehouse_Prague_1 = df_lagged[df_lagged["warehouse"] == "Prague_1"]

Prague_1_data_X = warehouse_Prague_1.drop(columns=["orders", "warehouse"])
Prague_1_data_y = warehouse_Prague_1["orders"]
Prague_1_data_X.tail()

,date,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14
1188,2024-03-11,10250.0,10344.0,11917.0,9754.0,9283.0,9316.0,9520.0,9900.0,10432.0,11683.0,9628.0,9078.0,9074.0,9776.0
1189,2024-03-12,9866.0,10250.0,10344.0,11917.0,9754.0,9283.0,9316.0,9520.0,9900.0,10432.0,11683.0,9628.0,9078.0,9074.0
1190,2024-03-13,9710.0,9866.0,10250.0,10344.0,11917.0,9754.0,9283.0,9316.0,9520.0,9900.0,10432.0,11683.0,9628.0,9078.0
1191,2024-03-14,9628.0,9710.0,9866.0,10250.0,10344.0,11917.0,9754.0,9283.0,9316.0,9520.0,9900.0,10432.0,11683.0,9628.0
1192,2024-03-15,10056.0,9628.0,9710.0,9866.0,10250.0,10344.0,11917.0,9754.0,9283.0,9316.0,9520.0,9900.0,10432.0,11683.0


In [28]:
warehouse_Prague_1_test = df_test[df_test["warehouse"] == "Prague_1"].drop(columns=["warehouse"])
concatinated_train_test_Prague_1 = pd.concat([Prague_1_data_X, warehouse_Prague_1_test])

In [29]:
date_encoder = DateToOrdinal()
concatinated_train_test_Prague_1["date"] = date_encoder.fit_transform(concatinated_train_test_Prague_1["date"].copy())


,date,t-1,t-2,t-3,t-4,t-5,t-6,t-7,t-8,t-9,t-10,t-11,t-12,t-13,t-14
56,1235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,1236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,1237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,1238,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,1239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
concatinated_train_test_Prague_1 = concatinated_train_test_Prague_1.reset_index(drop=True)

In [47]:
starting_date = len(Prague_1_data_X)

# Step 1: Selecting the specific row
row = concatinated_train_test_Prague_1.loc[starting_date - 1]

# Step 2: Excluding the "date" column
row_excluding_date = row.loc[row.index != 'date']

# Step 3: Shifting the data
shifted_row = row_excluding_date.shift()

shifted_row["t-1"] = Prague_1_data_y.tail(1)

shifted_row


/tmp/ipykernel_6245/1962320828.py:12: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  shifted_row["t-1"] = Prague_1_data_y.tail(1)


t-1     11917.0
t-2     10056.0
t-3      9628.0
t-4      9710.0
t-5      9866.0
t-6     10250.0
t-7     10344.0
t-8     11917.0
t-9      9754.0
t-10     9283.0
t-11     9316.0
t-12     9520.0
t-13     9900.0
t-14    10432.0
Name: 1178, dtype: float64